In [ ]:
! pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

In [3]:
! sudo apt update

[sudo] password for pms: 


In [ ]:
! sudo apt-get install poppler-utils

In [ ]:
! sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn

In [ ]:
! pip install unstructured-pytesseract
! pip install tesseract-ocr

In [1]:
from unstructured.partition.pdf import partition_pdf

/home/pms/llm_project/multimodal_rag_project01/bot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_pdf_elements=partition_pdf(
    filename="/home/pms/llm_project/multimodal_rag_project01/data/cj.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data",
)
     

In [ ]:
raw_pdf_elements

In [3]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
img=[]
tab=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))
  elif "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))
  elif "unstructured.documents.elements.Table" in str(type(element)):
            tab.append(str(element))

In [4]:
tab

['Type your email... Subscribe']

In [7]:
img=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))

In [9]:
img[2]

'EV/NTM EV/2024 EV/NTM NTMRev Gross Operating FCF % in Top 10 Company Rev Rev FCF Growth Margin Margin Margin Multiple LTM 1 Snowflake 15.5x 13.4x 55x 27% 66% (41%) 25% 100% 2 MongoDB 14.6x 12.9x 133x 17% 74% (18%) 3% 67% 3 Palantir 14.5x 13.9x 58x 19% 80% 2% 22% 47% 4 Cloudflare 13.4x 12.6x 153x 28% 76% (16%) 8% 100% 5 Datadog 13.1x 12.4x 52x 19% 80% (5%) 25% 99% 6 CrowdStrike 12.5x 11.1x 37x 31% 74% (6%) 30% 31% 7 Adobe 12.3x 11.9x 30x 12% 88% 34% 40% 26% 8 ServiceNow 12.2x 11.6x 38x 21% 79% 8% 28% 70% 9 Samsara 11.8x 10.5x 393x 31% 72% (34%) (3%) 72% 10 Zscaler 11.8x 10.5x 48x 27% 78% (14%) 21% 36% Overall Median = 50x. 48x 33.7 18% = 75% = (16%) 8% Clouded Judgement @jaminball ALTIMETER'

In [9]:
raw_pdf_elements2=partition_pdf(
    filename="/home/pms/llm_project/multimodal_rag_project01/data2/nlp-tasks-paper.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data2",
)

In [ ]:
! pip install langchain-core

In [ ]:
! pip install langchain
! pip install chromadb

In [5]:
%pip install -qU langchain-mistralai

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import json
config_data = json.load(open("./config.json"))
MISTRALAI_API_KEY = config_data["MISTRAL_API_KEY"]
GROQ_API_KEY = config_data["GROQ_API_KEY"]
os.environ["MISTRALAI_API_KEY"] = MISTRALAI_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [16]:
os.environ["GROQ_API_KEY"]

'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'

In [ ]:
! pip install langchain-groq

In [18]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llava-v1.5-7b-4096-preview",
    temperature=0.5,
    api_key=GROQ_API_KEY,
    max_tokens=4096,
    max_retries=2
)

In [8]:
from langchain_mistralai import MistralAIEmbeddings
embeddings = MistralAIEmbeddings(
    model="mistral-embed"
)

/home/pms/llm_project/multimodal_rag_project01/bot/lib/python3.10/site-packages/langchain_mistralai/embeddings.py:175: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
#from langchain_openai import ChatOpenAI

## Text Summary

In [11]:
# Prompt
prompt_text = """You are an assistant tasked with summarizing text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text elements. \
    Give a concise summary of the table or text that is well optimized for retrieval.text: {element} """


In [13]:
prompt = ChatPromptTemplate.from_template(prompt_text)

In [20]:
summarize_chain = {"element": lambda x: x} | prompt | llm | StrOutputParser()

In [22]:
text_summaries = []
text_summaries = summarize_chain.batch(Text,{"max_concurrency":5})

In [23]:
text_summaries

['2',
 '"10Y" refers to a 10-year investment, and the return on investment is 4.6%.',
 'The given text does not provide any context or information for me to summarize. Please provide a specific text for me to assist with summarizing.',
 'The table is titled "Table of Contents" and contains information on various topics. The first section is titled "Introduction" and provides an overview of the topics covered in the table. The second section is titled "Topic 1" and provides a more detailed explanation of the first topic. The third section is titled "Topic 2" and provides a similar explanation of the second topic. The table continues in this manner, with each section providing a detailed explanation of a different topic. The table is well-organized and easy to read, making it an effective tool for navigation and understanding the content.']

## Table Summary

In [25]:
table_summaries = []
table_summaries = summarize_chain.batch(tab, {"max_concurrency": 5})

In [26]:
table_summaries

['Email address: [type your email here] Subscribe to receive updates and news.']